## GOAL
Load helper functions for plots and simulation.


### What you should notice
We reuse the same helper functions for all visuals.


In [ ]:
# Import QuantumCircuit to build Bell and superdense circuits.
from qiskit import QuantumCircuit
# Import Statevector to inspect exact simulator states when needed.
from qiskit.quantum_info import Statevector
# Import StatevectorSampler for lightweight, exact sampling (no Aer).
from qiskit.primitives import StatevectorSampler
# Import histogram plotting for measurement results.
from qiskit.visualization import plot_histogram
# Import display to show circuit diagrams in VS Code.
from IPython.display import display
# Import matplotlib to force plot rendering.
import matplotlib.pyplot as plt

# Create a simulator sampler for all measurements in this notebook.
sampler = StatevectorSampler()

# Helper: print the statevector (useful for small circuits).
def show_statevector(circuit_no_measure, label='Statevector'):
    # Convert the circuit to a statevector before measurement.
    sv = Statevector.from_instruction(circuit_no_measure)
    # Print a label for clarity.
    print(f'\n--- {label} ---')
    # Print amplitudes for basis states like |00>, |01>, |10>, |11>.
    print(sv)
    # Return the statevector for further use if needed.
    return sv

# Helper: run a measured circuit and plot the histogram.
def run_and_plot_counts(circuit_with_measure, shots=500, title='Histogram'):
    # Run the circuit with the sampler and chosen number of shots.
    result = sampler.run([circuit_with_measure], shots=shots).result()
    # Extract counts from the result (measured bitstrings).
    counts = result[0].data.meas.get_counts()
    # Print counts so students can see the raw numbers.
    print(f'\nCounts (shots={shots}):', counts)
    # Plot the histogram of measurement results.
    plot_histogram(counts, title=title)
    # Force the plot to render in VS Code.
    plt.show()
    # Return counts for later calculations.
    return counts


## EXERCISE
No exercise here. Continue to the next cell.


In [ ]:
# Student solution cell (optional).


## GOAL
Create a Bell state and observe simulator counts.


### What you should notice
Only 00 and 11 appear with high probability.


In [ ]:
# Build a 2-qubit Bell circuit with measurement.
bell = QuantumCircuit(2, 2)
# Put qubit 0 into superposition.
bell.h(0)
# Entangle qubit 0 with qubit 1.
bell.cx(0, 1)
# Measure both qubits into classical bits.
bell.measure([0, 1], [0, 1])
# Display the circuit diagram.
display(bell.draw(output='mpl'))
plt.show()
# Run and plot counts to see correlated results.
run_and_plot_counts(bell, shots=500, title='Bell state counts')


## EXERCISE
Compute the correlation rate: (00 + 11) / total.


In [ ]:
# Student solution cell (optional).


## GOAL
Superdense coding: send 2 classical bits using 1 qubit + entanglement.


### What you should notice
Each message decodes to a unique bitstring.

Encoding table:
- 00 -> I
- 01 -> X
- 10 -> Z
- 11 -> XZ


In [ ]:
# Build a superdense coding circuit for a given 2-bit message.
def superdense(msg):
    # Start with a Bell pair shared between Alice (q0) and Bob (q1).
    qc = QuantumCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    # Alice encodes her two-bit message on qubit 0.
    if msg == '01':
        qc.x(0)
    elif msg == '10':
        qc.z(0)
    elif msg == '11':
        qc.x(0)
        qc.z(0)
    # Bob decodes using CX then H.
    qc.cx(0, 1)
    qc.h(0)
    # Measure both qubits to read the 2-bit message.
    qc.measure([0, 1], [0, 1])
    return qc

# Test all four messages and show histograms.
for msg in ['00', '01', '10', '11']:
    qc = superdense(msg)
    display(qc.draw(output='mpl'))
    plt.show()
    run_and_plot_counts(qc, shots=500, title=f'Superdense message {msg}')


## EXERCISE
Test all four messages and verify the decoded output.


In [ ]:
# Student solution cell (optional).


## GOAL
Optional: Run on a real IBM QPU (end of workshop).


### What you should notice
Real hardware results are noisy and queues are normal. Simulator output is the baseline.

You need:
- An IBM Quantum account
- QISKIT_IBM_TOKEN set in your environment
- Optional install: pip install qiskit-ibm-runtime


In [ ]:
# Optional QPU run (only here do we import IBM Runtime).
import os
# Import IBM Runtime tools for real hardware execution.
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
# Import pass manager for hardware-aware transpilation.
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Read the IBM token from the environment.
token = os.getenv('QISKIT_IBM_TOKEN')
# Stop if the token is missing.
if not token:
    raise SystemExit('Token missing. Set QISKIT_IBM_TOKEN and restart the kernel.')

# Create the runtime service connection.
service = QiskitRuntimeService(channel='ibm_quantum', token=token)
# Choose a real, least-busy backend.
backend = service.least_busy(simulator=False, operational=True)
print('Using backend:', backend.name)

# Transpile the Bell circuit for the selected backend.
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
bell_t = pm.run(bell)

# Run the job on hardware with SamplerV2.
sampler_qpu = SamplerV2(backend=backend)
job = sampler_qpu.run([bell_t], shots=1024)
print('Job ID:', job.job_id())
print('Queue delays are normal. Compare to simulator baseline.')
# Fetch results when complete.
result = job.result()
# Convert quasi-probabilities to approximate counts.
quasi = result.quasi_dists[0]
counts = {k: int(v * 1024) for k, v in quasi.items()}
# Plot the QPU histogram.
plot_histogram(counts, title='QPU Bell counts')
plt.show()


## EXERCISE
Optional: compare QPU counts to simulator counts and describe differences.


In [ ]:
# Student solution cell (optional).
